In [ ]:
import os, time, pathlib, pprint, requests, json
from datetime import datetime, timedelta

EP = os.getenv("BROWSER_ENDPOINT", "http://browser:8004")
SCRAPED = pathlib.Path("/storage/scraped_data")

def wait_for(job_id, every=3):
    print(f"⏳ Waiting for job {job_id}...")
    while True:
        rec = requests.get(f"{EP}/jobs/{job_id}").json()
        status = rec["status"]
        if status not in {"finished", "error"}:
            print(f"\r⏱️  {rec['status_with_elapsed']}", end="")
        else:
            print(f"\n✅ {status.upper()}")
            return rec
        time.sleep(every)

def submit(task, payload):
    print(f"🚀 Submitting {task} task...")
    print(f"📝 Payload: {json.dumps(payload, indent=2)}")
    r = requests.post(f"{EP}/jobs/{task}", json=payload)
    r.raise_for_status()
    jid = r.json()["job_id"]
    print(f"🆔 Job ID: {jid}")
    return wait_for(jid)

def analyze_results(result, test_name):
    print(f"\n" + "="*60)
    print(f"📊 ANALYSIS: {test_name}")
    print("="*60)
    
    if result["status"] == "error":
        print(f"❌ FAILED: {result.get('error', 'Unknown error')}")
        return
    
    if "result" in result and result["result"].get("success"):
        res = result["result"]
        print(f"✅ SUCCESS: Found {res.get('hotels_found', 0)} hotels")
        print(f"📍 Location: {res.get('location')}")
        print(f"📅 Dates: {res.get('date_range')}")
        print(f"🏨 Nights: {res.get('nights')}")
        if res.get('average_price_per_night'):
            print(f"💰 Avg Price: ${res['average_price_per_night']}/night")
        if res.get('average_rating'):
            print(f"⭐ Avg Rating: {res['average_rating']}/10")
        print(f"⏱️  Execution Time: {res.get('execution_time_seconds')}s")
        print(f"📁 Data File: {res.get('data_file')}")
    else:
        print(f"❌ Task failed or returned unexpected result")
        pprint.pp(result)

# Calculate future dates for testing
today = datetime.now()
check_in_1 = (today + timedelta(days=30)).strftime("%Y-%m-%d")
check_out_1 = (today + timedelta(days=33)).strftime("%Y-%m-%d")
check_in_2 = (today + timedelta(days=45)).strftime("%Y-%m-%d")
check_out_2 = (today + timedelta(days=47)).strftime("%Y-%m-%d")
check_in_3 = (today + timedelta(days=60)).strftime("%Y-%m-%d")
check_out_3 = (today + timedelta(days=64)).strftime("%Y-%m-%d")

print(f"📅 Test dates calculated:")
print(f"   Test 1: {check_in_1} to {check_out_1}")
print(f"   Test 2: {check_in_2} to {check_out_2}")
print(f"   Test 3: {check_in_3} to {check_out_3}")

📅 Test dates calculated:
   Test 1: 2025-09-15 to 2025-09-18
   Test 2: 2025-09-30 to 2025-10-02
   Test 3: 2025-10-15 to 2025-10-19


In [ ]:
# Test Case 1: Basic search (Riyadh hotels)
basic_search = {
    "location": "Riyadh, Saudi Arabia",
    "check_in": check_in_1,
    "check_out": check_out_1,
    "adults": 2,
    "max_results": 10
}

result_1 = submit("booking-hotels", basic_search)
analyze_results(result_1, "Basic Hotel Search - Riyadh")

🚀 Submitting booking-hotels task...
📝 Payload: {
  "location": "Riyadh, Saudi Arabia",
  "check_in": "2025-09-15",
  "check_out": "2025-09-18",
  "adults": 2,
  "max_results": 10
}
🆔 Job ID: 7eecc96019c14c80a76793bf4b71e789
⏳ Waiting for job 7eecc96019c14c80a76793bf4b71e789...
⏱️  running 16m 21s
✅ FINISHED

📊 ANALYSIS: Basic Hotel Search - Riyadh
✅ SUCCESS: Found 10 hotels
📍 Location: Riyadh, Saudi Arabia
📅 Dates: 2025-09-15 to 2025-09-18
🏨 Nights: 3
⏱️  Execution Time: 982.4s
📁 Data File: hotels_data.json


In [ ]:
# Test Case 2: Budget hotels with filters
budget_search = {
    "location": "Dubai, UAE",
    "check_in": check_in_2,
    "check_out": check_out_2,
    "adults": 1,
    "max_price": 300,
    "min_rating": 7.0,
    "max_results": 15
}

result_2 = submit("booking-hotels", budget_search)
analyze_results(result_2, "Budget Hotels - Dubai")

🚀 Submitting booking-hotels task...
📝 Payload: {
  "location": "Dubai, UAE",
  "check_in": "2025-09-30",
  "check_out": "2025-10-02",
  "adults": 1,
  "max_price": 300,
  "min_rating": 7.0,
  "max_results": 15
}
🆔 Job ID: d47ddbab0fac48a2a8f7bf1a0c2a68a3
⏳ Waiting for job d47ddbab0fac48a2a8f7bf1a0c2a68a3...
⏱️  running 1m 48s
✅ FINISHED

📊 ANALYSIS: Budget Hotels - Dubai
❌ Task failed or returned unexpected result
{'job_id': 'd47ddbab0fac48a2a8f7bf1a0c2a68a3',
 'task_name': 'booking-hotels',
 'params': {'location': 'Dubai, UAE',
            'check_in': '2025-09-30',
            'check_out': '2025-10-02',
            'adults': 1,
            'max_price': 300,
            'min_rating': 7.0,
            'max_results': 15},
 'status': 'finished',
 'created_at': '2025-08-16T18:43:10.351660',
 'started_at': '2025-08-16T18:43:10.351877',
 'finished_at': '2025-08-16T18:44:59.774507',
 'result': {'success': False,
            'error': 'Search form submission timed out - Booking.com may be '
   

In [ ]:
# Test Case 3: Luxury family trip
luxury_search = {
    "location": "Jeddah, Saudi Arabia",
    "check_in": check_in_3,
    "check_out": check_out_3,
    "adults": 2,
    "children": 2,
    "rooms": 2,
    "min_price": 500,
    "star_rating": [4, 5],
    "amenities": ["pool", "wifi", "gym"],
    "max_results": 20
}

result_3 = submit("booking-hotels", luxury_search)
analyze_results(result_3, "Luxury Family Trip - Jeddah")

In [ ]:
# Test Case 4: Business trip (no reviews needed)
business_search = {
    "location": "KAUST, Thuwal, Saudi Arabia",
    "check_in": check_in_1,
    "check_out": check_out_1,
    "adults": 1,
    "amenities": ["wifi"],
    "include_reviews": False,
    "max_results": 5
}

result_4 = submit("booking-hotels", business_search)
analyze_results(result_4, "Business Trip - KAUST Area")

🚀 Submitting booking-hotels task...
📝 Payload: {
  "location": "KAUST, Thuwal, Saudi Arabia",
  "check_in": "2025-09-15",
  "check_out": "2025-09-18",
  "adults": 1,
  "amenities": [
    "wifi"
  ],
  "include_reviews": false,
  "max_results": 5
}
🆔 Job ID: b7d52c57ed814604933f7dc07fd8d959
⏳ Waiting for job b7d52c57ed814604933f7dc07fd8d959...
⏱️  running 1m 24s
✅ FINISHED

📊 ANALYSIS: Business Trip - KAUST Area
❌ Task failed or returned unexpected result
{'job_id': 'b7d52c57ed814604933f7dc07fd8d959',
 'task_name': 'booking-hotels',
 'params': {'location': 'KAUST, Thuwal, Saudi Arabia',
            'check_in': '2025-09-15',
            'check_out': '2025-09-18',
            'adults': 1,
            'amenities': ['wifi'],
            'include_reviews': False,
            'max_results': 5},
 'status': 'finished',
 'created_at': '2025-08-16T18:46:18.743146',
 'started_at': '2025-08-16T18:46:18.743375',
 'finished_at': '2025-08-16T18:47:43.667761',
 'result': {'success': False,
            

In [ ]:
# Test Case 5: Conference attendee scenario (proximity search)
conference_search = {
    "location": "Riyadh International Convention Center, Riyadh",
    "search_radius": "5km",
    "check_in": check_in_2,
    "check_out": check_out_2,
    "adults": 1,
    "max_results": 15
}

result_5 = submit("booking-hotels", conference_search)
analyze_results(result_5, "Conference Proximity Search")

In [ ]:
# Test Summary
print("📋 BOOKING HOTELS TASK - TEST SUMMARY")
print("="*60)

test_results = [
    ("Basic Search - Riyadh", result_1),
    ("Budget Hotels - Dubai", result_2),
    ("Luxury Family - Jeddah", result_3),
    ("Business Trip - KAUST", result_4),
    ("Conference Proximity", result_5)
]

successful_tests = 0
failed_tests = 0
total_hotels = 0
total_execution_time = 0

for test_name, result in test_results:
    if result["status"] == "finished" and result.get("result", {}).get("success"):
        successful_tests += 1
        hotels_found = result["result"].get("hotels_found", 0)
        exec_time = result["result"].get("execution_time_seconds", 0)
        total_hotels += hotels_found
        total_execution_time += exec_time
        print(f"✅ {test_name}: {hotels_found} hotels ({exec_time}s)")
    else:
        failed_tests += 1
        error_msg = result.get("error", "Unknown error")
        print(f"❌ {test_name}: FAILED - {error_msg}")

print("\n📊 OVERALL STATISTICS:")
print(f"   ✅ Successful tests: {successful_tests}/{len(test_results)}")
print(f"   ❌ Failed tests: {failed_tests}/{len(test_results)}")
print(f"   🏨 Total hotels collected: {total_hotels}")
print(f"   ⏱️  Total execution time: {total_execution_time:.1f}s")
print(f"   📁 Data files saved to: {SCRAPED}/booking-hotels/")

if successful_tests == len(test_results):
    print("\n🎉 ALL TESTS PASSED! The booking-hotels task is working perfectly.")
elif successful_tests >= len(test_results) * 0.75:
    print("\n✅ Most tests passed. The booking-hotels task is working well with minor issues.")
else:
    print("\n⚠️  Multiple tests failed. The booking-hotels task needs debugging.")

print("\n🔍 To examine detailed results, check the JSON files in the data directories.")